In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stat
import statsmodels.api as sm
import statsmodels.formula.api as smf#, ols
from sklearn.preprocessing import OneHotEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import ttest_ind, f_oneway
import scipy.stats as stats
import sys
from pathlib import Path
# # Add src directory to the Python path
sys.path.append(str(Path.cwd().parent))
from src.data_analysis.df_dataAttribute_analysis import get_data_attributes
from src.data_sourcing.import_export_data import get_data, save_data
from src.data_analysis.df_ttest_anova_analysis import run_one_sample_ttest, run_univariate_tests
from src.data_analysis.df_multicollinearity_analysis import compute_vif, compute_vif_vectorized
from src.data_analysis.df_correlation_analysis import compute_correlations
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
df = get_data('featured_engineered_aggregations.csv', 'processed_data')
df.head(5)

,ATL_OR_DR,CAMPAIGN_TYPE,CHANNEL,COST,FREE_TRIALS,HOLIDAY_FLAG,SEASON,MONTH,ATL_OR_DR_CAMPAIGN_TYPE,ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL,...,FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_CAMPAIGN_TYPE_MONTH,AVG_COST_PER_FREE_TRIALS_BY_CAMPAIGN_TYPE_MONTH,MEDIAN_COST_PER_FREE_TRIALS_BY_CAMPAIGN_TYPE_MONTH,COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TRIALS_BY_CAMPAIGN_TYPE_MONTH,AVG_COST_BY_CAMPAIGN_TYPE_MONTH,MEDIAN_COST_BY_CAMPAIGN_TYPE_MONTH,COST_OVER_AVG_COST_BY_CAMPAIGN_TYPE_MONTH,NORMALIZE_FREE_TRIAL_BY_MONTH,NORMALIZE_COST_PER_FREE_TRIALS_BY_MONTH,NORMALIZE_COST_BY_FREE_TRIALS_BY_MONTH
0,DR - Direct Response,Title,paid social,7784.31,86401.15,False,Summer,June,DR - Direct Response_Title,DR - Direct Response_Title_paid social,...,2.161878,0.942693,0.129226,0.095572,12783.161145,3432.160,0.608950,2.015642,0.086220,0.701295
1,DR - Direct Response,Title,app,2474.31,2956.74,False,Summer,August,DR - Direct Response_Title,DR - Direct Response_Title_app,...,0.070544,0.853002,0.107078,0.981049,14949.239779,3239.655,0.165514,0.067269,1.023058,0.013036
2,ATL - Above The Line,Title,paid social,10222.82,12513.08,False,Summer,July,ATL - Above The Line_Title,ATL - Above The Line_Title_paid social,...,0.263725,0.850812,0.101840,0.960225,13099.998683,2802.460,0.780368,0.272262,0.984393,0.226908
3,DR - Direct Response,Title,app,49631.87,17207.04,False,Winter,December,DR - Direct Response_Title,DR - Direct Response_Title_app,...,0.367143,0.871586,0.109911,3.309360,14484.916524,3133.110,3.426452,0.362288,2.645155,0.979314
4,ATL - Above The Line,Title,paid social,2081.06,21758.33,False,Summer,August,ATL - Above The Line_Title,ATL - Above The Line_Title_paid social,...,0.519126,0.853002,0.107078,0.112127,14949.239779,3239.655,0.139208,0.495028,0.116928,0.080683


### Multi-collinearity Testing Analysis

In [38]:
temp_formula=['MONTH',
'ATL_OR_DR_MONTH',
'CAMPAIGN_TYPE_MONTH',
'CHANNEL_MONTH',
'SEASON',
'LOG_FREE_TRIALS',
'LOG_COST_BY_FREE_TRIALS',
'NORMALIZE_FREE_TRIAL_BY_MONTH',
'NORMALIZE_COST_BY_FREE_TRIALS_BY_MONTH']


In [46]:
# df.columns

# X_df= df.drop(columns=['FREE_TRIALS', 'ATL_OR_DR_CAMPAIGN_TYPE', 'ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL', 'ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL_MONTH',
#                          'ATL_OR_DR_CAMPAIGN_TYPE_MONTH', 'ATL_OR_DR_CHANNEL_MONTH', 'CAMPAIGN_TYPE_CHANNEL_MONTH', 'CAMPAIGN_TYPE_MONTH', 'CHANNEL_MONTH',
#                            'LOG_FREE_TRIALS', 'ATL_OR_DR_MONTH'], axis=1)
# X_df.columns
#
hashmap = get_data_attributes(df)
formula = hashmap['numerical']
formula.remove('FREE_TRIALS')
formula.remove('LOG_FREE_TRIALS')
formula.remove('LOG_COST')
formula.extend(['CHANNEL', 'HOLIDAY_FLAG', 'SEASON']) #'CHANNEL'
print("formula:", formula)
temp_df = df[temp_formula]
vif_result = compute_vif_vectorized(#compute_vif(
    data=temp_df,
)
vif_result
# '''
# Try an aggregation calculation by Channel and month? So 2 group by's?
# '''
vif_result[(vif_result['VIF']  <10) | (vif_result['VIF'] == float('inf'))]

formula: ['COST', 'COST_PER_FREE_TRIALS', 'COST_BY_FREE_TRIALS', 'LOG_COST_PER_FREE_TRIALS', 'LOG_COST_BY_FREE_TRIALS', 'AVG_COST_MONTH', 'MEDIAN_COST_MONTH', 'AVG_COST_CAMPAIGN_TYPE_CHANNEL', 'MEDIAN_COST_CAMPAIGN_TYPE_CHANNEL', 'AVG_FREE_TRIALS_MONTH', 'MEDIAN_FREE_TRIALS_MONTH', 'AVG_FREE_TRIALS_BY_CHANNEL_MONTH', 'MEDIAN_FREE_TRIALS_BY_CHANNEL_MONTH', 'FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_CHANNEL_MONTH', 'AVG_COST_PER_FREE_TRIALS_BY_CHANNEL_MONTH', 'MEDIAN_COST_PER_FREE_TRIALS_BY_CHANNEL_MONTH', 'COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TRIALS_BY_CHANNEL_MONTH', 'AVG_COST_BY_CHANNEL_MONTH', 'MEDIAN_COST_BY_CHANNEL_MONTH', 'COST_OVER_AVG_COST_BY_CHANNEL_MONTH', 'AVG_FREE_TRIALS_BY_ATL_OR_DR_MONTH', 'MEDIAN_FREE_TRIALS_BY_ATL_OR_DR_MONTH', 'FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_ATL_OR_DR_MONTH', 'AVG_COST_PER_FREE_TRIALS_BY_ATL_OR_DR_MONTH', 'MEDIAN_COST_PER_FREE_TRIALS_BY_ATL_OR_DR_MONTH', 'COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TRIALS_BY_ATL_OR_DR_MONTH', 'AVG_COST_BY_ATL_OR_D

,Feature,VIF
109,CAMPAIGN_TYPE_MONTH_Title_April,3.911164e+00
110,CHANNEL_MONTH_paid social_April,3.616340e+00
111,ATL_OR_DR_MONTH_DR - Direct Response_April,3.110302e+00
112,LOG_FREE_TRIALS,2.674354e+00
113,CHANNEL_MONTH_display_April,2.586861e+00
...,...,...
258,ATL_OR_DR_MONTH_ATL - Above The Line_December,-2.458941e+14
259,SEASON_Summer,-2.495192e+14
260,MONTH_October,-2.918808e+14
261,CAMPAIGN_TYPE_MONTH_Title_August,-4.138636e+14


In [8]:
numerical_hashset = set(list(vif_result[(vif_result['VIF'] > 10) | (vif_result['VIF'] == float('inf'))]['Feature']))

In [9]:
numerical_hashset

{'AVG_COST_MONTH',
 'COST',
 'COST_BY_FREE_TRIALS',
 'COST_OVER_AVG_COST_BY_ATL_OR_DR_MONTH',
 'COST_OVER_AVG_COST_BY_CAMPAIGN_TYPE_MONTH',
 'COST_OVER_AVG_COST_BY_CHANNEL_MONTH',
 'COST_PER_FREE_TRIALS',
 'COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TRIALS_BY_ATL_OR_DR_MONTH',
 'COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TRIALS_BY_CAMPAIGN_TYPE_MONTH',
 'FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_ATL_OR_DR_MONTH',
 'FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_CAMPAIGN_TYPE_MONTH',
 'FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_CHANNEL_MONTH',
 'MEDIAN_FREE_TRIALS_MONTH',
 'NORMALIZE_COST_BY_FREE_TRIALS_BY_MONTH',
 'NORMALIZE_COST_PER_FREE_TRIALS_BY_MONTH',
 'NORMALIZE_FREE_TRIAL_BY_MONTH'}

### One Sample T-Test Aanalysis

In [10]:
categorical_list = ['ATL_OR_DR','HOLIDAY_FLAG']
quantitative = ["FREE_TRIALS", "LOG_FREE_TRIALS"]
avg_free_trials = round(df['FREE_TRIALS'].mean(),2)
avg_log_free_trials = round(df['LOG_FREE_TRIALS'].mean(),2)
# 2) Using a dictionary for different hypothesized means per column:
results_2 = run_one_sample_ttest(
    df=df,
    categorical_cols=categorical_list,
    quantitative_cols=quantitative,
    popmean={"FREE_TRIALS": avg_free_trials, "LOG_FREE_TRIALS": avg_log_free_trials},  # different popmeans
    stat_type="mean",
    alpha=0.05,  # more stringent significance level
    sort_results=True
)
results_2

,Categorical Variable,Category,Quantitative Variable,Sample Size,Sample Mean,T-statistic,p-value,Decision,Error
0,HOLIDAY_FLAG,True,LOG_FREE_TRIALS,534,10.201621,-0.637220,0.524255,Fail to Reject H0,None
1,ATL_OR_DR,ATL - Above The Line,LOG_FREE_TRIALS,8601,10.228078,-0.178557,0.858290,Fail to Reject H0,None
2,ATL_OR_DR,DR - Direct Response,LOG_FREE_TRIALS,5507,10.227902,-0.155741,0.876243,Fail to Reject H0,None
3,ATL_OR_DR,DR - Direct Response,FREE_TRIALS,5507,45547.216388,0.112888,0.910123,Fail to Reject H0,None
4,HOLIDAY_FLAG,False,LOG_FREE_TRIALS,13574,10.229048,-0.111226,0.911438,Fail to Reject H0,None
5,ATL_OR_DR,ATL - Above The Line,FREE_TRIALS,8601,45401.434358,-0.085614,0.931775,Fail to Reject H0,None
6,HOLIDAY_FLAG,True,FREE_TRIALS,534,45484.437959,0.009366,0.992530,Fail to Reject H0,None
7,HOLIDAY_FLAG,False,FREE_TRIALS,13574,45457.313076,-0.001986,0.998415,Fail to Reject H0,None


### ANOVA Testing Analysis

In [11]:
# https://towardsdatascience.com/anova-test-with-python-cfbf4013328b
hashmap = get_data_attributes(df)
categorical = hashmap['categorical']
categorical.remove('ATL_OR_DR')
categorical.remove('HOLIDAY_FLAG')
categorical

['CAMPAIGN_TYPE',
 'CHANNEL',
 'SEASON',
 'MONTH',
 'ATL_OR_DR_CAMPAIGN_TYPE',
 'ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL',
 'ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL_MONTH',
 'ATL_OR_DR_CAMPAIGN_TYPE_MONTH',
 'ATL_OR_DR_CHANNEL',
 'ATL_OR_DR_CHANNEL_MONTH',
 'ATL_OR_DR_MONTH',
 'CAMPAIGN_TYPE_CHANNEL',
 'CAMPAIGN_TYPE_CHANNEL_MONTH',
 'CAMPAIGN_TYPE_MONTH',
 'CHANNEL_MONTH']

In [12]:
quantitative = ["FREE_TRIALS", "LOG_FREE_TRIALS"]

anova_results_df = run_univariate_tests(
    df=df,
    categorical_cols=categorical,
    quantitative_cols=quantitative,
    test_type="anova"
)
anova_results_df[anova_results_df['p-value'] <= 0.50]

,Categorical Variable,Quantitative Variable,F-statistic,p-value,Error
0,MONTH,FREE_TRIALS,1.745376,0.057730,None
1,ATL_OR_DR_MONTH,FREE_TRIALS,1.437204,0.080345,None
2,CAMPAIGN_TYPE_MONTH,FREE_TRIALS,1.346628,0.104410,None
3,CHANNEL_MONTH,FREE_TRIALS,1.096844,0.170871,None
4,MONTH,LOG_FREE_TRIALS,1.335306,0.197357,None
5,CAMPAIGN_TYPE_CHANNEL_MONTH,FREE_TRIALS,1.061845,0.206772,None
6,ATL_OR_DR_CAMPAIGN_TYPE_MONTH,FREE_TRIALS,1.112372,0.268880,None
7,ATL_OR_DR_CHANNEL_MONTH,FREE_TRIALS,1.050398,0.285439,None
8,ATL_OR_DR_MONTH,LOG_FREE_TRIALS,1.085444,0.352263,None
9,ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL_MONTH,FREE_TRIALS,1.023266,0.361860,None


In [13]:
categorical_hashset = set(list(anova_results_df[anova_results_df['p-value'] <= 0.50]['Categorical Variable']))

### Kruskal Wallis Testing Analysis

In [14]:
kruskal_results_df = run_univariate_tests(
    df=df,
    categorical_cols=categorical,
    quantitative_cols=quantitative,
    test_type="kruskal"
)
kruskal_results_df[kruskal_results_df['p-value'] <= 0.60]

,Categorical Variable,Quantitative Variable,H-statistic,p-value,Error
0,MONTH,FREE_TRIALS,15.421565,0.163999,None
1,MONTH,LOG_FREE_TRIALS,15.421565,0.163999,None
2,ATL_OR_DR_MONTH,LOG_FREE_TRIALS,26.406948,0.282034,None
3,ATL_OR_DR_MONTH,FREE_TRIALS,26.406948,0.282034,None
4,SEASON,FREE_TRIALS,2.744503,0.432717,None
5,SEASON,LOG_FREE_TRIALS,2.744503,0.432717,None
6,CHANNEL_MONTH,LOG_FREE_TRIALS,193.675803,0.493063,None
7,CHANNEL_MONTH,FREE_TRIALS,193.675803,0.493063,None
8,ATL_OR_DR_CHANNEL_MONTH,LOG_FREE_TRIALS,233.572700,0.586903,None
9,ATL_OR_DR_CHANNEL_MONTH,FREE_TRIALS,233.572700,0.586903,None


In [15]:
if 'ATL_OR_DR_MONTH' in categorical_hashset:
    print("hi")

hi


### Pearson Correlation Testing Analysis

In [16]:
# 1) Pearson correlation (numeric only - only linear relationships)
pearson_results = compute_correlations(
    df=df,
    include_categorical=False,
    cat_cols=None  # Do not encode categorical variables
)
pearson_results[
    (pearson_results['p-value'] < 0.20) &
    (pearson_results['Variable 1'] == 'FREE_TRIALS')
]

,Variable 1,Variable 2,Correlation,p-value,Method,Error
100,FREE_TRIALS,FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_CHANNEL_MONTH,0.972843,0.000000e+00,pearson,None
113,FREE_TRIALS,FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_CAMPAIGN_T...,0.995560,0.000000e+00,pearson,None
127,FREE_TRIALS,FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_ATL_OR_DR_...,0.996181,0.000000e+00,pearson,None
152,FREE_TRIALS,LOG_COST_BY_FREE_TRIALS,0.347763,0.000000e+00,pearson,None
153,FREE_TRIALS,LOG_FREE_TRIALS,0.747446,0.000000e+00,pearson,None
158,FREE_TRIALS,NORMALIZE_FREE_TRIAL_BY_MONTH,0.997894,0.000000e+00,pearson,None
192,FREE_TRIALS,NORMALIZE_COST_BY_FREE_TRIALS_BY_MONTH,0.230571,1.356582e-169,pearson,None
201,FREE_TRIALS,COST_BY_FREE_TRIALS,0.219831,5.792956e-154,pearson,None
208,FREE_TRIALS,LOG_COST_PER_FREE_TRIALS,-0.215401,9.361971e-148,pearson,None
291,FREE_TRIALS,AVG_FREE_TRIALS_BY_CHANNEL_MONTH,0.122735,1.752471e-48,pearson,None


In [17]:
pearson_hashmap = set(list(pearson_results[(pearson_results['p-value'] < 0.05) &(pearson_results['Variable 1'] == 'FREE_TRIALS')]['Variable 2']))
len(pearson_hashmap)

25

In [18]:
numerical_hashset = pearson_hashmap - numerical_hashset
print(len(numerical_hashset))

14


### Spearman Correlation Testing Analysis

In [19]:
# 2) Spearman correlation (with categorical one-hot encoding)
spearman_results = compute_correlations(
    df=df,
    include_categorical=True,  # Encode categorical variables
    drop_first=True,
    cat_cols=categorical
)
spearman_results[(spearman_results['Variable 1'] == 'FREE_TRIALS') & (spearman_results['p-value'] < 0.05)]

#spearman_results[((spearman_results['Variable 1'] == 'FREE_TRIALS') | (spearman_results['Variable 1'] == 'LOG_FREE_TRIALS')) & (spearman_results['p-value'] < 0.05)]

,Variable 1,Variable 2,Correlation,p-value,Method,Error
130,FREE_TRIALS,COST_PER_FREE_TRIALS,-0.460507,0.000000e+00,spearman,None
131,FREE_TRIALS,COST_BY_FREE_TRIALS,0.470630,0.000000e+00,spearman,None
133,FREE_TRIALS,LOG_FREE_TRIALS,1.000000,0.000000e+00,spearman,None
134,FREE_TRIALS,LOG_COST_PER_FREE_TRIALS,-0.460507,0.000000e+00,spearman,None
135,FREE_TRIALS,LOG_COST_BY_FREE_TRIALS,0.470630,0.000000e+00,spearman,None
141,FREE_TRIALS,FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_CHANNEL_MONTH,0.986150,0.000000e+00,spearman,None
144,FREE_TRIALS,COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TR...,-0.449598,0.000000e+00,spearman,None
145,FREE_TRIALS,COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TR...,-0.459360,0.000000e+00,spearman,None
148,FREE_TRIALS,FREE_TRIALS_OVER_AVG_FREE_TRIALS_BY_CAMPAIGN_T...,0.997544,0.000000e+00,spearman,None
152,FREE_TRIALS,COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TR...,-0.458958,0.000000e+00,spearman,None


In [20]:
spearman_hashset = set(list(spearman_results[(spearman_results['Variable 1'] == 'FREE_TRIALS') & (spearman_results['p-value'] < 0.05)]['Variable 2']))
numerical_hashset = numerical_hashset | spearman_hashset
len(numerical_hashset)

28

In [21]:
numerical_hashset.remove('LOG_FREE_TRIALS')

### Random Testing Analysis

In [55]:
def prepare_and_run_ols(categorical_vars_df, quantitative_vars_df, target_variable, drop_first=True):
    # Create copy to avoid modifying original data
    categorical_vars = categorical_vars_df.copy()
    quantitative_vars = quantitative_vars_df.copy()

    # Initialize encoder
    encoder = OneHotEncoder(drop='first' if drop_first else None, sparse_out=False)

    # Fit and transform categorical variables
    categorical_encoded = encoder.fit_transform(categorical_vars)

    # Get feature names for encoded variables
    encoded_feature_names = []
    for i, feature in enumerate(categorical_vars.columns):
        cats = encoder.categories_[i]
        if drop_first:
            cats = cats[1:]  # Skip first category if dropping
        encoded_feature_names.extend([f"{feature}_{cat}" for cat in cats])

    # Create DataFrame with encoded categorical variables
    categorical_df = pd.DataFrame(
        categorical_encoded,
        columns=encoded_feature_names,
        index=categorical_vars.index
    )

    # Combine categorical and quantitative variables
    X = pd.concat([categorical_df, quantitative_vars], axis=1)

    # Add constant for intercept
    X = sm.add_constant(X)

    # Fit OLS model
    model = sm.OLS(target_variable, X)
    results = model.fit()

    # Calculate VIF scores
    vif_data = calculate_vif(X)

    return results, vif_data

def calculate_vif(X):
    vif_df = pd.DataFrame()
    vif_df["Variable"] = X.columns
    vif_df["VIF"] = [variance_inflation_factor(X.values, i)
                     for i in range(X.shape[1])]
    return vif_df.sort_values('VIF', ascending=False)

'\n# Your data should already be in DataFrame format:\ncategorical_vars_df = df[[\'ATL_OR_DR\', \'CAMPAIGN_TYPE\', \'CHANNEL\', \'HOLIDAY_FLAG\', \'MONTH\']]\nquantitative_vars_df = df[[\'COST\', \'COST_PER_FREE_TRIALS\', \'COST_BY_FREE_TRIALS\']]\ntarget = df[\'FREE_TRIALS\']\n\n# Run the model\nmodel_results, vif_scores = prepare_and_run_ols(\n    categorical_vars_df=categorical_vars_df,\n    quantitative_vars_df=quantitative_vars_df,\n    target_variable=target\n)\n\n# Print results\nprint(model_results.summary())\nprint("\nVariance Inflation Factors:")\nprint(vif_scores)\n'

In [56]:
'''
1. Model Fit and Overfitting
    Extremely High R² (≈ 99.75%)
        The model explains nearly all the variance in your target. Although on the surface this looks excellent, such high R² in an initial, highly complex model (with many engineered features) is a red flag for overfitting. Overfitting is likely because the model is “learning” the noise and the collinearities inherent in your training data.

    Adjusted R² Close to R²
        Even the adjusted R² is nearly as high as the raw R², which is common when you have a huge number of predictors. However, when predictors are highly redundant, adjusted R² can be misleading.

2. Multicollinearity Concerns
    Small Eigenvalue and VIFs
        The smallest eigenvalue is extremely close to zero (1.27e-14), and the VIF table shows some predictors with VIF values on the order of 10^13. This is a clear signal of severe multicollinearity. In practical terms, many of your predictors are nearly linear combinations of others.

    Implications
        With multicollinearity:
        The individual coefficients become unstable and their standard errors inflate.
        P-values for many features may not be reliable.
        It becomes difficult to assess the individual contribution of a given feature because changes in one predictor may be almost perfectly mirrored by changes in another.

3. Feature Engineering and Redundancy
    Many Engineered Features
        Your feature set includes many dummies (e.g., for months, channels, and campaign types) and interactions/aggregations among them. This can lead to a large number of predictors that overlap in the information they provide.

    Suggestion
        Consider reviewing groups of features that are similar in nature. For example, if you have many month‐by‐channel interaction terms that are all very highly correlated with each other, it might be wise to either:
            Combine them (or use a summary statistic), or
            Use a regularization technique that “shrinks” redundant coefficients (such as LASSO or Ridge).

4. Coefficient Interpretations and p-Values
    Coefficient Size and Significance
        Some coefficients are extremely large (or small), and many have p-values of 0.000. However, because of the severe multicollinearity, the individual significance tests (t-tests) are not very trustworthy. When predictors are collinear, even if a variable looks “significant,” it may not be uniquely contributing to the model.

    What to Do
        Rather than rely solely on p-values, consider:
            Using regularized regression methods (LASSO, Ridge, or Elastic Net) that can perform feature selection and mitigate multicollinearity.
            Running stepwise or other variable‐selection procedures (with caution) to help determine which predictors add incremental predictive value.

5. Data Preparation: Standardization and Train–Test Splitting
    Standardization
        You have not yet standardized your features. Without standardization, variables measured on different scales can lead to coefficient estimates that are not directly comparable. Standardizing (or otherwise scaling) the predictors can help when you later use methods that are sensitive to the scale (like Ridge regression) and can also aid in interpreting effect sizes.

    Train–Test Split
        The current model is run on the full dataset. To assess true predictive performance and avoid data leakage, it is essential to:
            Split the data into training and test sets.
            Re-run feature engineering (ensuring no leakage) on the training set, and then apply the same transformations to the test set.
            Evaluate model performance on unseen data.

6. Recommendations for Feature Selection
    Drop or Combine Highly Redundant Features
        Given the astronomical VIFs, you should investigate the features that are nearly linear combinations. It may be helpful to:
            Use domain knowledge to choose a subset of the most meaningful features.
            Consider dimensionality reduction techniques (e.g., Principal Component Analysis) if you need to retain information from many correlated features.
    Regularization Methods
        Techniques such as LASSO (which can zero out coefficients) or Ridge (which can shrink them) can be very useful when you have many correlated predictors. These methods also tend to improve out‐of‐sample prediction performance by reducing overfitting.

    Examine Aggregated and Interaction Terms
        Look carefully at the engineered interactions and aggregations. If some of these features are “repeats” of the same underlying signal (e.g., different ways of summarizing month effects), consider selecting the most interpretable or robust ones.
 '''
# Recap:
# The model is too complex and over‐parameterized.
# Many features are redundant and should be pruned or combined.
# It is critical to apply standardization and use proper cross-validation (or train–test splits) in future iterations.
categorical_vars = list(categorical_hashset)
numerical_vars = list(numerical_hashset)
categorical_vars = df[categorical_vars]
quantitative_vars = df[numerical_vars]
target = df['FREE_TRIALS']

# Run the model
model_results, vif_scores = prepare_and_run_ols(
    categorical_vars_df=categorical_vars,
    quantitative_vars_df=quantitative_vars,
    target_variable=target
)

# Print summary
print(model_results.summary())

# Check VIF scores
print("\nVariance Inflation Factors:")
print(vif_scores)

# Access specific statistics
print("\nR-squared:", model_results.rsquared)
print("Adjusted R-squared:", model_results.rsquared_adj)
print("F-statistic p-value:", model_results.f_pvalue)

c:\Users\cheng\Workspace\Paramount\.venv\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\cheng\Workspace\Paramount\.venv\lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


                            OLS Regression Results                            
Dep. Variable:            FREE_TRIALS   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                 1.250e+04
Date:                Fri, 31 Jan 2025   Prob (F-statistic):               0.00
Time:                        18:33:45   Log-Likelihood:            -1.3302e+05
No. Observations:               14108   AIC:                         2.669e+05
Df Residuals:                   13668   BIC:                         2.703e+05
Df Model:                         439                                         
Covariance Type:            nonrobust                                         
                                                                                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------

In [65]:
temp_df = pd.concat([categorical_vars, quantitative_vars], axis=1)
temp_df

,ATL_OR_DR_MONTH,CHANNEL,CAMPAIGN_TYPE_MONTH,ATL_OR_DR_CAMPAIGN_TYPE_MONTH,CAMPAIGN_TYPE,SEASON,ATL_OR_DR_CHANNEL_MONTH,ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL_MONTH,CAMPAIGN_TYPE_CHANNEL_MONTH,CAMPAIGN_TYPE_CHANNEL,...,NORMALIZE_COST_BY_FREE_TRIALS_BY_MONTH,AVG_COST_BY_CAMPAIGN_TYPE_MONTH,MEDIAN_FREE_TRIALS_BY_CAMPAIGN_TYPE_MONTH,MEDIAN_COST_BY_CAMPAIGN_TYPE_MONTH,COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TRIALS_BY_CAMPAIGN_TYPE_MONTH,COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TRIALS_BY_CHANNEL_MONTH,AVG_FREE_TRIALS_BY_ATL_OR_DR_MONTH,AVG_COST_PER_FREE_TRIALS_BY_CHANNEL_MONTH,MEDIAN_COST_PER_FREE_TRIALS_BY_ATL_OR_DR_MONTH,MEDIAN_COST_PER_FREE_TRIALS_BY_CHANNEL_MONTH
0,DR - Direct Response_June,paid social,Title_June,DR - Direct Response_Title_June,Title,Summer,DR - Direct Response_paid social_June,DR - Direct Response_Title_paid social_June,Title_paid social_June,Title_paid social,...,0.701295,12783.161145,24130.56,3432.160,0.095572,0.061628,41278.932086,1.461924,0.119202,0.127788
1,DR - Direct Response_August,app,Title_August,DR - Direct Response_Title_August,Title,Summer,DR - Direct Response_app_August,DR - Direct Response_Title_app_August,Title_app_August,Title_app,...,0.013036,14949.239779,26403.99,3239.655,0.981049,1.354346,48221.595657,0.617890,0.100050,0.124839
2,ATL - Above The Line_July,paid social,Title_July,ATL - Above The Line_Title_July,Title,Summer,ATL - Above The Line_paid social_July,ATL - Above The Line_Title_paid social_July,Title_paid social_July,Title_paid social,...,0.226908,13099.998683,27828.02,2802.460,0.960225,0.905088,45883.862370,0.902643,0.118229,0.126064
3,DR - Direct Response_December,app,Title_December,DR - Direct Response_Title_December,Title,Winter,DR - Direct Response_app_December,DR - Direct Response_Title_app_December,Title_app_December,Title_app,...,0.979314,14484.916524,28693.37,3133.110,3.309360,5.082883,46403.302137,0.567472,0.135247,0.136529
4,ATL - Above The Line_August,paid social,Title_August,ATL - Above The Line_Title_August,Title,Summer,ATL - Above The Line_paid social_August,ATL - Above The Line_Title_paid social_August,Title_paid social_August,Title_paid social,...,0.080683,14949.239779,26403.99,3239.655,0.112127,0.106248,41227.896600,0.900200,0.110240,0.119401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14103,ATL - Above The Line_February,bvod,Brand_February,ATL - Above The Line_Brand_February,Brand,Winter,ATL - Above The Line_bvod_February,ATL - Above The Line_Brand_bvod_February,Brand_bvod_February,Brand_bvod,...,0.045338,9435.677520,27127.84,2749.450,0.165003,0.077298,41183.020972,1.129223,0.127209,0.134706
14104,ATL - Above The Line_April,ooh,Title_April,ATL - Above The Line_Title_April,Title,Spring,ATL - Above The Line_ooh_April,ATL - Above The Line_Title_ooh_April,Title_ooh_April,Title_ooh,...,0.005861,12780.401689,27959.54,2680.580,0.231480,0.304455,45219.071625,0.481326,0.099473,0.077285
14105,ATL - Above The Line_December,bvod,Title_December,ATL - Above The Line_Title_December,Title,Winter,ATL - Above The Line_bvod_December,ATL - Above The Line_Title_bvod_December,Title_bvod_December,Title_bvod,...,7.739032,14484.916524,28693.37,3133.110,6.384853,4.973122,47938.062271,1.119006,0.108830,0.126611
14106,ATL - Above The Line_February,paid social,Brand_February,ATL - Above The Line_Brand_February,Brand,Winter,ATL - Above The Line_paid social_February,ATL - Above The Line_Brand_paid social_February,Brand_paid social_February,Brand_paid social,...,3.853152,9435.677520,27127.84,2749.450,6.367959,3.424029,41183.020972,0.983829,0.127209,0.093300


In [66]:
vif_result_2 = compute_vif(temp_df)
vif_result_2

,Feature,VIF
0,CHANNEL_MONTH_digital audio_January,1.699472e+14
1,CHANNEL_MONTH_ooh_January,4.372427e+13
2,ATL_OR_DR_CHANNEL_MONTH_ATL - Above The Line_o...,2.595735e+13
3,CAMPAIGN_TYPE_CHANNEL_MONTH_Brand_digital audi...,2.566154e+13
4,ATL_OR_DR_CHANNEL_MONTH_DR - Direct Response_p...,2.421290e+13
...,...,...
1410,NORMALIZE_COST_BY_FREE_TRIALS_BY_MONTH,3.377013e+01
1411,COST_BY_FREE_TRIALS,3.309037e+01
1412,COST_PER_FREE_TRIALS_OVER_AVG_COST_PER_FREE_TR...,6.345738e+00
1413,LOG_COST_PER_FREE_TRIALS,4.316289e+00


In [67]:
save_data(vif_result_2, 'vif_results.csv')

2025-01-31 23:44:08,293 - INFO - Data saved successfully!
Environment: development
Data Type: processed_data
File Path: C:\Users\cheng\Workspace\Paramount\data\processed\vif_results.csv
Rows Saved: 1415


In [82]:

def compute_vif_vectorized(data: pd.DataFrame, drop_first: bool = True) -> pd.DataFrame:
    """
    Compute Variance Inflation Factor (VIF) for all numeric and one-hot encoded features
    using a vectorized approach via the inverse of the correlation matrix.

    Parameters:
        data (pd.DataFrame): The input DataFrame.
        drop_first (bool): Whether to drop the first category during one-hot encoding.

    Returns:
        pd.DataFrame: A DataFrame with two columns: 'Feature' and 'VIF', sorted by VIF descending.
    """
    # Step 1: Copy the data
    df = data.copy()

    # Step 2: One-hot encode categorical variables (if any)
    cat_cols = df.select_dtypes(include=["object", "category", "bool"]).columns
    if len(cat_cols) > 0:
        df = pd.get_dummies(df, columns=cat_cols, drop_first=drop_first)

    # Step 3: Ensure all features are numeric and handle non-numeric entries
    df = df.apply(pd.to_numeric, errors='coerce')

    # Step 4: Replace infinite values and drop rows with NaNs
    df = df.replace([np.inf, -np.inf], np.nan).dropna()

    # Ensure all values are floats (this helps with numerical stability)
    df = df.astype(float)

    # (Optional) If you were previously adding a constant for regression,
    # you typically do NOT include the constant when computing VIF.
    # Thus, we leave it out here.

    # Step 5: Compute the correlation matrix
    corr_matrix = df.corr()

    # Step 6: Invert the correlation matrix
    # If the matrix is nearly singular, use the pseudo-inverse
    try:
        inv_corr_matrix = np.linalg.inv(corr_matrix.values)
    except np.linalg.LinAlgError:
        inv_corr_matrix = np.linalg.pinv(corr_matrix.values)

    # Step 7: Extract VIFs from the diagonal of the inverse correlation matrix
    vif_values = np.diag(inv_corr_matrix)

    # Step 8: Format the results in a DataFrame
    vif_df = pd.DataFrame({
        "Feature": corr_matrix.columns,
        "VIF": vif_values
    }).sort_values(by="VIF", ascending=False).reset_index(drop=True)

    return vif_df


In [83]:
vif_result_201 = compute_vif_vectorized(temp_df)

In [85]:
save_data(vif_result_201, 'vif_results_201.csv')

2025-01-31 23:52:01,621 - INFO - Data saved successfully!
Environment: development
Data Type: processed_data
File Path: C:\Users\cheng\Workspace\Paramount\data\processed\vif_results_201.csv
Rows Saved: 1415


In [28]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from statsmodels.stats.outliers_influence import variance_inflation_factor

def prepare_and_run_ols_with_pca(categorical_vars_df, quantitative_vars_df, target_variable,
                                 drop_first=True, variance_threshold=0.90):
    # --- Step 1: Preprocessing ---
    # Copy to avoid modifying original data
    categorical_vars = categorical_vars_df.copy()
    quantitative_vars = quantitative_vars_df.copy()

    # Initialize one-hot encoder (note: in recent versions of sklearn, use `sparse=False`)
    encoder = OneHotEncoder(drop='first' if drop_first else None, sparse=False)

    # Fit and transform categorical variables
    categorical_encoded = encoder.fit_transform(categorical_vars)

    # Generate column names for the encoded variables
    encoded_feature_names = []
    for i, feature in enumerate(categorical_vars.columns):
        cats = encoder.categories_[i]
        print("cats:", cats)
        if drop_first:
            cats = cats[1:]  # Skip the first category if dropping
            print("new-cats:", cats)
        encoded_feature_names.extend([f"{feature}_{cat}" for cat in cats])
        print("encoded_feature_names:", encoded_feature_names)
    # Create a DataFrame for the encoded categorical variables
    categorical_encoded_df = pd.DataFrame(categorical_encoded, columns=encoded_feature_names,
                                          index=categorical_vars.index)

    # Combine the encoded categorical and quantitative predictors
    X = pd.concat([categorical_encoded_df, quantitative_vars], axis=1)

    # --- Step 2: Standardize the predictors ---
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # --- Step 3: Run PCA ---
    # Using PCA with n_components as a float will select the number of components
    # such that the cumulative explained variance is at least the given threshold.
    pca = PCA(n_components=variance_threshold, svd_solver='full')
    X_pca = pca.fit_transform(X_scaled)

    # Create a DataFrame for the principal components
    pc_columns = [f'PC{i+1}' for i in range(X_pca.shape[1])]
    X_pca_df = pd.DataFrame(X_pca, columns=pc_columns, index=X.index)

    # --- Step 4: Prepare for and run OLS ---
    # Add constant for the intercept
    X_pca_df = sm.add_constant(X_pca_df)

    # Fit OLS using the principal components
    model = sm.OLS(target_variable, X_pca_df)
    results = model.fit()

    # --- Step 5: Compute VIF (for completeness) ---
    # Note: PCA components are orthogonal, so VIFs should be ~1.
    vif_data = calculate_vif(X_pca_df)

    return results, vif_data, pca.explained_variance_ratio_

def calculate_vif(X):
    """
    Compute Variance Inflation Factor (VIF) for each feature in DataFrame X.
    """
    vif_df = pd.DataFrame()
    vif_df["Variable"] = X.columns
    vif_df["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_df.sort_values('VIF', ascending=False)


In [29]:
categorical_vars = list(categorical_hashset)
numerical_vars = list(numerical_hashset)
categorical_vars = df[categorical_vars]
quantitative_vars = df[numerical_vars]
target = df['FREE_TRIALS']

model_results, vif_scores, pca_results = prepare_and_run_ols_with_pca(
    categorical_vars_df=categorical_vars,
    quantitative_vars_df=quantitative_vars,
    target_variable=target
)


c:\Users\cheng\Workspace\Paramount\.venv\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


cats: ['ATL - Above The Line_April' 'ATL - Above The Line_August'
 'ATL - Above The Line_December' 'ATL - Above The Line_February'
 'ATL - Above The Line_January' 'ATL - Above The Line_July'
 'ATL - Above The Line_June' 'ATL - Above The Line_March'
 'ATL - Above The Line_May' 'ATL - Above The Line_November'
 'ATL - Above The Line_October' 'ATL - Above The Line_September'
 'DR - Direct Response_April' 'DR - Direct Response_August'
 'DR - Direct Response_December' 'DR - Direct Response_February'
 'DR - Direct Response_January' 'DR - Direct Response_July'
 'DR - Direct Response_June' 'DR - Direct Response_March'
 'DR - Direct Response_May' 'DR - Direct Response_November'
 'DR - Direct Response_October' 'DR - Direct Response_September']
new-cats: ['ATL - Above The Line_August' 'ATL - Above The Line_December'
 'ATL - Above The Line_February' 'ATL - Above The Line_January'
 'ATL - Above The Line_July' 'ATL - Above The Line_June'
 'ATL - Above The Line_March' 'ATL - Above The Line_May'
 'ATL 

In [30]:
model_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            FREE_TRIALS   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.949
Method:                 Least Squares   F-statistic:                     772.8
Date:                Sat, 01 Feb 2025   Prob (F-statistic):               0.00
Time:                        12:25:58   Log-Likelihood:            -1.5411e+05
No. Observations:               14108   AIC:                         3.089e+05
Df Residuals:                   13764   BIC:                         3.115e+05
Df Model:                         343                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.546e+04    114.391    397.394      0.000    4.52e+04    4.57e+04
PC1          871.9585     32.112     27.153      0.000     809.014     934.903
PC2         -109.0681     35.167     -3.101      0.002    -178.000     -40.136
PC3          534.6086     36.982     14.456      0.000     462.119     607.098
PC4          258.7153     37.578      6.885      0.000     185.057     332.374
PC5          -47.2472     38.064     -1.241      0.215    -121.858      27.364
PC6           -3.2777     39.093     -0.084      0.933     -79.905      73.350
PC7         -128.9803     39.542     -3.262      0.001    -206.487     -51.473
PC8           65.3740     39.605      1.651      0.099     -12.256     143.004
PC9          -11.4340     39.806     -0.287      0.774     -89.460      66.592
PC10          68.0259     39.883      1.706      0.088     -10.150     146.201
PC11        -254.9172     40.873     -6.237      0.000    -335.034    -174.800
PC12         851.4605     42.606     19.985      0.000     767.947     934.974
PC13        -604.5454     44.011    -13.736      0.000    -690.812    -518.279
PC14       -1324.6768     44.401    -29.835      0.000   -1411.708   -1237.645
PC15        3620.7556     44.763     80.887      0.000    3533.013    3708.498
PC16           4.0222     44.966      0.089      0.929     -84.118      92.162
PC17        -317.4422     45.078     -7.042      0.000    -405.802    -229.082
PC18        1430.3301     45.455     31.467      0.000    1341.233    1519.427
PC19         115.5058     45.593      2.533      0.011      26.137     204.875
PC20         486.8267     45.831     10.622      0.000     396.991     576.662
PC21        -366.0459     45.923     -7.971      0.000    -456.062    -276.030
PC22        -711.9177     46.000    -15.476      0.000    -802.085    -621.751
PC23        -955.9397     46.166    -20.707      0.000   -1046.431    -865.448
PC24        -867.0109     46.240    -18.750      0.000    -957.648    -776.373
PC25         117.9520     46.323      2.546      0.011      27.153     208.752
PC26          15.9295     46.364      0.344      0.731     -74.950     106.809
PC27         350.4818     46.445      7.546      0.000     259.443     441.521
PC28          55.8178     46.551      1.199      0.231     -35.429     147.065
PC29        -858.8473     46.652    -18.410      0.000    -950.292    -767.403
PC30        1161.4746     46.883     24.774      0.000    1069.578    1253.371
PC31        -358.2693     46.972     -7.627      0.000    -450.341    -266.197
PC32       -1260.7498     47.235    -26.691      0.000   -1353.337   -1168.162
PC33       -1300.6284     47.482    -27.392      0.000   -1393.699   -1207.557
PC34       -1090.1796     47.766    -22.823      0.000   -1183.807    -996.552
PC35         496.4201     48.022     10.337      0.000     402.291     590.549
PC

In [63]:
# (-1, 1) => (row, column) where -1 is set to auto for either row or column.
np.array(categorical_vars).reshape(-1, 1)

array([['DR - Direct Response_June'],
       ['paid social'],
       ['Title_June'],
       ...,
       ['Title_youtube'],
       ['February'],
       ['youtube_February']], dtype=object)